In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import shap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

In [2]:
data = pd.read_csv('../Cleaned_full_data.csv')

C:\Users\adi_s\AppData\Local\Temp\ipykernel_1992\3121833706.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Cleaned_full_data.csv')


In [4]:
# Reset retained index.
data = data.reset_index(drop=True)
# Set NA to 0.
data['ct_ftp_cmd'] = data['ct_ftp_cmd'].fillna(0)
data['attack_cat'] = data['attack_cat'].str.replace(r'\s+', '', regex=True)
data['attack_cat'] = data['attack_cat'].str.replace('Backdoors', 'Backdoor')

In [6]:
data = data.drop(columns=['proto', 'dsport', 'service', 'state', 'srcip', 'sport', 'dstip'])


In [8]:
features = data.drop(columns=['attack_cat', 'label'])
label = data['label']
attack_cat = data['attack_cat']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)
scaled_data_df = pd.DataFrame(scaled_data, columns=features.columns)
scaled_data_df['attack_cat'] = attack_cat.reset_index(drop=True)
# Split 20% test.

In [5]:
ohe1 = pd.read_csv('DoS_proto_encoded.csv')
ohe2 = pd.read_csv('DoS_dsport_encoded.csv')
ohe3 = pd.read_csv('DoS_service_encoded.csv')
ohe4 = pd.read_csv('DoS_state_encoded.csv')
ohe5 = pd.read_csv('DoS_srcip_encoded.csv')
ohe6 = pd.read_csv('DoS_sport_encoded.csv')
ohe7 = pd.read_csv('DoS_dstip_encoded.csv')

In [9]:
full_data = pd.concat([data, ohe1, ohe2, ohe3, ohe4, ohe5, ohe6, ohe7], axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data_df, label, test_size=0.2, random_state=42)
# Drop attack cat before running model and store for later evaluation indexing.
test_attack_cat = X_test['attack_cat']
X_train = X_train.drop(columns=['attack_cat'])
X_test = X_test.drop(columns=['attack_cat'])
# Check label distribution.
print(scaled_data_df['attack_cat'].value_counts())

attack_cat
Normal            2218764
Generic            215481
Exploits            44525
Fuzzers             24246
DoS                 16353
Reconnaissance      13987
Analysis             2677
Backdoor             2329
Shellcode            1511
Worms                 174
Name: count, dtype: int64


In [12]:
# Run RandomForest with default settings.
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    443831
           1       0.98      0.98      0.98     64179

    accuracy                           1.00    508010
   macro avg       0.99      0.99      0.99    508010
weighted avg       1.00      1.00      1.00    508010

[[442862    969]
 [  1135  63044]]
